# Earthquake Calendar

Given the two datasets provided on Slack in class, write a program to output a calendar of when earthquakes occurred. Your program should be callable as such:

```python
# for example, print the earthquake calendar for February 2024.
print(earthquake_calendar(2024, 2))   
```

You should use the `create_full_calendar` function below as a base. Display an `X` instead of the date if no earthquakes occurred on that date.

Prepare this for _both_ datasets. _(Hint: the `time` field in the smaller dataset is some type of numeric representation for a time... maybe a "timestamp")

In [42]:
import calendar
import pandas as pd
from datetime import datetime

earthquakes_2023_df = pd.read_csv("./earthquakes_2023_global.csv")
earthquakes_df = pd.read_csv("./earthquakes_dataset.csv") # Unix timestamp one
earthquake_dates = [] # List of dates for earthquakes

# Split the string of datetimestamp by delimiter 'T' and use the first half to make datetime value
# To make the above datetime value we also needed to supply the format of the date - in this case, thats 'Year-Month-Day' or '%Y-%m-%d'
# Now we can extract the date from the datetime value via datetime.date() to store within our container of dates 'earthquake_dates'
for time in earthquakes_2023_df.loc[(earthquakes_2023_df["type"] == "earthquake")]["time"]:
    earthquake_dates.append(datetime.strptime(time.split("T")[0], "%Y-%m-%d").date())

# Figured out how to convert from unix timestamp to datetime via:
# https://stackoverflow.com/questions/3682748/converting-unix-timestamp-string-to-readable-date
# Important that we convert the milliseconds timestamp to a seconds timestamp by dividing by 1000
# This is because datetime.utcfromtimestamp only takes timestamps in the format of seconds, while our dataset has it stored in milliseconds
# Other than the coversion from unix timestamp to datetime, do exact same thing as above
for time in earthquakes_df.loc[(earthquakes_df["type"] == "earthquake")]["time"]:
    earthquake_dates.append(datetime.utcfromtimestamp(float(time) / 1000).date())

def earthquake_calendar(year, month):
    first_weekday, days_in_month = calendar.monthrange(year, month)
    
    day_counter = 1
    output_str = f"    {calendar.month_name[month]} {year}\n"
    output_str += "Su Mo Tu We Th Fr Sa\n"
    
    # Add initial spacing for the first week
    output_str += "   " * first_weekday
    
    while day_counter <= days_in_month:
        for weekday in range(7):
            if day_counter > days_in_month:
                break  # Break the loop if the last day is reached

            # Figured out fastest way to check if list contains a certain value via:
            # https://stackoverflow.com/questions/7571635/fastest-way-to-check-if-a-value-exists-in-a-list
            # Use local date variables to make datetime value
            # Then we can extract the date from said value to see if the date is within our 'earthquake_dates' container
            is_earthquake_day = datetime(year=year, month=month, day=day_counter).date() in earthquake_dates

            # Print day number if earthquake occured on that day
            if is_earthquake_day:
                output_str += f"{day_counter:2d} "
            # Otherwise print X
            else:
                output_str += f" X "
            
            # Check if we need to move to the next line
            if (first_weekday + day_counter) % 7 == 0:
                output_str += "\n"
            
            day_counter += 1
    
    return output_str

# Create and print the full calendar
print(earthquake_calendar(2023, 12))


    December 2023
Su Mo Tu We Th Fr Sa
             1  2  3 
 4  5  6  7  8  9 10 
11 12 13 14 15 16 17 
18 19 20 21 22 23 24 
25 26 27 28 29  X  X 

